[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/betloreilly/hybridsearch_llmevaluation/blob/main/Hybrid%20Search%20and%20LLM%20Evaluation.ipynb)

## Astra DB's Hybrid Search and LLM Evaluation

The goal of this tutorial is to demonstrate how to build a system that can efficiently identify and evaluate the top job opportunities for candidates.

We will build together a sample end-to-end use case that will be able to find and evaluate the best job opportunities for candidates by using the power of LLM and Astra Hybrid Search capabilities.


## Install Dependencies

In [1]:
!pip install python-dotenv langchain openai sentence-transformers cassio tiktoken python-dotenv
!pip install cassandra-driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.3 MB/s eta 0:00:00
  C

## Importing neccessary libraries and organizing configuration

<!-- Here you can find the `conf.env` template:
>   ```
>   ASTRA_API_KEY =
>   ASTRA_SECURE_CONNECT_BUNDLE_PATH =
>   OPENAI_API_KEY=  
>   ``` -->


In [4]:
# Config
import os
import pandas as pd
import numpy as np
import json
from getpass import getpass

from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT, ProtocolVersion
from cassandra.auth import PlainTextAuthProvider
from langchain.vectorstores import Cassandra
from langchain.schema.document import Document

from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

profile = ExecutionProfile(request_timeout=30)

# Set parameters for Astra DB connection
os.environ["ASTRA_API_KEY"] = getpass('Enter your Astra DB API Key/Token ("AstraCS:..."): ')
os.environ["ASTRA_SECURE_CONNECT_BUNDLE_PATH"] = ''
os.environ["ASTRA_DB_KEYSPACE"] = input('Enter your Astra DB Keyspace name (e.g. vector): ')
os.environ["ASTRA_DB_TABLE_NAME"] = input('Enter your Astra DB Table name (e.g. jobs): ')
os.environ["OPENAI_API_KEY"] = getpass('Enter your OpenAI API key: ')

#assert os.environ.get("ASTRA_DB_KEYSPACE"), "⚠️ Please provide Astra DB keyspace name"
#assert os.environ.get("ASTRA_DB_TABLE_NAME"), "⚠️ Please provide Astra DB table name"


Enter your Astra DB API Key/Token ("AstraCS:..."): ··········
Enter your Astra DB Keyspace name (e.g. vector): vector
Enter your Astra DB Table name (e.g. jobs): jobs2
Enter your OpenAI API key: ··········


In [3]:
# Upload your Astra DB secure connect bundle
import os
from google.colab import files

print('Please upload your Astra DB Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Astra DB\'s Secure Connect Bundle. Please re-run the cell.'
    )

Please upload your Astra DB Secure Connect Bundle


Saving secure-connect-vector2.zip to secure-connect-vector2.zip


## Prepare table schema

Here is the DDL commands to get our schema up and running for later use.
You can find the table and index creation scripts below:
```
CREATE TABLE vector.jobs (
  job_id text PRIMARY KEY,
  job_title text,
  skills text,
  salary text,
  location text,
  embedding_vector vector<float, 1536> )

CREATE CUSTOM INDEX IF NOT EXISTS ann_index
  ON vector.jobs(embedding_vector) USING 'StorageAttachedIndex';

CREATE CUSTOM INDEX ix_location ON vector.jobs(Location ) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex' WITH OPTIONS = {
'index_analyzer': '{
        "tokenizer" : {"name" : "standard"},
        "filters" : [{"name" : "porterstem"}]
}'};


CREATE CUSTOM INDEX ix_salary ON vector.jobs(salary) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex' WITH OPTIONS = {
'index_analyzer': '{
        "tokenizer" : {"name" : "standard"},
        "filters" : [{"name" : "porterstem"}]
}'};

```



In [43]:
jobs_table = "CREATE TABLE IF NOT EXISTS " + os.environ.get("ASTRA_DB_KEYSPACE") + "." + os.environ.get("ASTRA_DB_TABLE_NAME") + " (job_id text PRIMARY KEY, job_title text, skills text, salary text, location text,embedding_vector VECTOR<FLOAT, 1536>);"
jobs_table_embedding_vector_idx = "CREATE CUSTOM INDEX IF NOT EXISTS jobs_emb_vec_idx ON " + os.environ.get("ASTRA_DB_KEYSPACE") + "." + os.environ.get("ASTRA_DB_TABLE_NAME") + "(embedding_vector) USING 'StorageAttachedIndex';"
jobs_table_location_idx = "CREATE CUSTOM INDEX IF NOT EXISTS jobs_loc_idx ON " + os.environ.get("ASTRA_DB_KEYSPACE") + "." + os.environ.get("ASTRA_DB_TABLE_NAME") +" (location) USING 'StorageAttachedIndex' WITH OPTIONS = {'index_analyzer': '{\"tokenizer\" : {\"name\" : \"standard\"},\"filters\" : [{\"name\" : \"porterstem\"}]}'" + "};"
jobs_table_salary_idx = "CREATE CUSTOM INDEX IF NOT EXISTS jobs_sal_idx ON " + os.environ.get("ASTRA_DB_KEYSPACE")  + "."  + os.environ.get("ASTRA_DB_TABLE_NAME") + " (salary) USING 'StorageAttachedIndex' WITH OPTIONS = {'index_analyzer': '{\"tokenizer\" : {\"name\" : \"standard\"},\"filters\" : [{\"name\" : \"porterstem\"}]}'" + "};"


## Configure AstraDB connection

In [44]:

cluster = Cluster(
    cloud={
        "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
    },
    auth_provider=PlainTextAuthProvider(
        "token", os.environ.get("ASTRA_API_KEY")
    ),
    protocol_version=ProtocolVersion.V4,
    execution_profiles={EXEC_PROFILE_DEFAULT: profile},
)

session = cluster.connect()
session.execute("USE " + os.environ.get("ASTRA_DB_KEYSPACE") + ";")
session.execute(jobs_table)
session.execute(jobs_table_embedding_vector_idx)
session.execute(jobs_table_location_idx)
session.execute(jobs_table_salary_idx)

## Defining function for embedding texts

In [39]:
import openai
openai.api_key = os.environ.get("OPENAI_API_KEY")
def generate_embedding(text):
    model = "text-embedding-ada-002"
    response = openai.Embedding.create(model=model, input=text)
    return response.data[0]['embedding']

## Loading a CSV file into Astra vector database after creating embeddings for `job_description`

In [45]:
import csv
from cassandra.query import SimpleStatement
from cassandra.concurrent import execute_concurrent_with_args
count = 0
# Inser query for the prepared statement
inser_embedding_stmt = f"""INSERT INTO """ + os.environ.get("ASTRA_DB_KEYSPACE") + """.""" + os.environ.get("ASTRA_DB_TABLE_NAME") + """ (job_id, job_title, skills, salary, location, embedding_vector) VALUES (?, ?, ?, ?, ?, ? )"""
prepared = session.prepare(inser_embedding_stmt)

# Input the CSV file for job listing
input_csv_file = 'jobs.csv'
print('Please upload your jobs.csv file')
jobs_sample_data_uploaded = files.upload()

# Open the input CSV file for reading
try:
    with open(input_csv_file, 'r', newline='') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=';')
        next(csvreader)

        for row in csvreader:
            count += 1
            job_title     = ' '.join(row[0:1])
            skills        = ' '.join(row[2:3])
            salary        = ' '.join(row[3:4])
            location      = ' '.join(row[4:5])
            combined_text = ' '.join(row[1:2])
            print(count, row[0:1], row[2:3], row[3:4], row[4:5])
            embedding_res = generate_embedding(combined_text)
            session.execute(prepared, (str(count), job_title, skills, salary, location, embedding_res))

except FileNotFoundError:
    print(f"File '{input_csv_file}' not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

Please upload your jobs.csv file


Saving jobs.csv to jobs (2).csv
1 ['Software Engineer'] ['Java, Python, JavaScript, SQL, Agile'] ['Salary: $90,000 - $120,000 per year'] ['Location: San Francisco, CA']
2 ['Data Analyst'] ['SQL, Excel, Data Visualization, Statistics'] ['Salary: $70,000 - $90,000 per year'] ['Location: New York, NY']
3 ['Network Administrator'] ['Cisco, VPN, Network Security, Troubleshooting'] ['Salary: $75,000 - $100,000 per year'] ['Location: Los Angeles, CA']
4 ['UX Designer'] ['User Research, Wireframing, Prototyping, UI Design'] ['Salary: $80,000 - $110,000 per year'] ['Location: Austin, TX']
5 ['Marketing Manager'] ['Digital Marketing, SEO, Team Leadership, Analytics'] ['Salary: $85,000 - $120,000 per year'] ['Location: Chicago, IL']
6 ['DevOps Engineer'] ['Docker, Kubernetes, AWS, Jenkins'] ['Salary: $95,000 - $130,000 per year'] ['Location: Seattle, WA']
7 ['Graphic Designer'] ['Adobe Creative Suite, Graphic Design, Multimedia'] ['Salary: $60,000 - $80,000 per year'] ['Location: Miami, FL']
8 ['

## Defining a function that will be used for evaluating the results

It is using gpt-4 to evaluate the results after having the top similar results from Astra DB

In [46]:
def get_completion_from_messages(messages, model="gpt-4", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Testing with sample CVs to find the best possible job posting for a given CV



*   For each sample CV in the file, combine `text search` to look for `location` and get the best results for similarity between job_posting and CV in Astra DB
*   If `similarity_search score` is bigger than a threshold, send the results to LLM to get a new scoring to confirm that we have an ideal job for the candidate.

We have only one Marketin Manager position is in Chicago so there was no result for Bob Smith after `text search` for `Location`.


In [54]:
import csv
from cassandra.query import SimpleStatement
count = 0

input_csv_file = 'test_cvs.csv'
print('Please upload your test_cvs.csv file')
cv_sample_data_uploaded = files.upload()
ASTRA_DB_KEYSPACE = os.environ.get("ASTRA_DB_KEYSPACE")
ASTRA_DB_TABLE_NAME = os.environ.get("ASTRA_DB_TABLE_NAME")
try:
    with open(input_csv_file, 'r', newline='') as csvfile:
        csvreader = csv.reader(csvfile,delimiter=';' )
        next(csvreader)

        for row in csvreader:
            count += 1
            name = ' '.join(row[0:1])
            job_title=' '.join(row[1:2])
            cv = ' '.join(row[2:3])
            location = ' '.join(row[3:4])
            salary = ' '.join(row[4:5])
            print("Search similarity for this CV:",cv,"\n")
            embedding_res = generate_embedding(cv)
            query = SimpleStatement(f"SELECT job_id,job_title,skills,salary,location,embedding_vector,similarity_cosine(embedding_vector, {embedding_res}) as score  FROM {ASTRA_DB_KEYSPACE}.{ASTRA_DB_TABLE_NAME} where location: '{location}' ORDER BY embedding_vector ANN OF {embedding_res} LIMIT 3")
            print(name, location)
            res = session.execute(query )
            for row in res:
              res_job = row.job_title
              id = row.job_id
              if row.score > 0.91:
                  print('Result: Score',row.score,' Job_id ',id,' ',res_job,"\n")
                  messages =  [
                  {'role':'system', 'content':'You are a chatbot for giving scores for the result of a job posting and CV comparison that are sent in [].You will help eliminating the candidates that doesnt fit the role by ranking them close to 1.'},
                  {'role':'system', 'content':'You need to give a score between 1 and 10. If it is a good candidate for the job , you can give 10. '},
                  {'role':'system', 'content': "You should give a detailed explanation how you decide the ranking and give the ranking result as a number at the end."},
                  {'role':'user', 'content':f'[{cv}],[{res_job}]' } ]
                  response = get_completion_from_messages(messages, temperature=0)
                  print(response)
            if count ==5:
             break
            print("#########################################################")

except FileNotFoundError:
    print(f"File '{input_csv_file}' not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

Please upload your test_cvs.csv file


Saving test_cvs.csv to test_cvs (7).csv
Search similarity for this CV: I am an experienced Software Engineer with expertise in Java, Python, and JavaScript. I have a strong background in software development and have worked on various projects, including web applications and backend systems. My skills include database design, API development, and problem-solving. I am passionate about writing clean and maintainable code and enjoy working in agile teams to deliver high-quality software solutions. 

John Doe San Francisco
Result: Score 0.9286186695098877  Job_id  1   Software Engineer 

Based on the job posting for a Software Engineer and the CV provided, the candidate seems to be a strong match. The candidate has experience in the role and has demonstrated expertise in Java, Python, and JavaScript, which are often key languages for a Software Engineer. The candidate's experience in software development, including web applications and backend systems, is also a strong point. 

The candid